In [ ]:
import cv2
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow
import math


class skin_detection():
    def __init__(self, img_path, mode):
        self.img_path = img_path 
        self.mode = mode 
        self.folder_img_path = None 
        self.folder_save_path = None

    # convert ảnh từ 2D thành ảnh argb
    def convert_ARGB(self):
        img_bgr = cv2.imread(self.img_path)
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        img_argb = np.zeros((img_rgb.shape[0], img_rgb.shape[1], 4), dtype=np.float32)
        img_argb[:, :, :3] = img_rgb
        img_argb[:, :, 3] = 255

        return img_argb

    # convert argb sang YCbCr
    def convert_YCrCb(self):
        img_bgr = cv2.imread(self.img_path)
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        img_YCrCb = np.zeros((img_rgb.shape[0], img_rgb.shape[1], 3), dtype=np.float32)
        img_YCrCb[:, :, 0] = 0.299*img_rgb[:, :, 0] + 0.587*img_rgb[:, :, 1] + 0.114*img_rgb[:, :, 2]
        img_YCrCb[:, :, 2] = 128 - 0.168736*img_rgb[:, :, 0] - 0.331264*img_rgb[:, :, 1] + 0.5*img_rgb[:, :, 2]
        img_YCrCb[:, :, 1] = 128 + 0.5*img_rgb[:, :, 0] - 0.418688*img_rgb[:, :, 1] - 0.081312*img_rgb[:, :, 2]

        # Chuyển đổi kiểu dữ liệu của ảnh thành uint8
        img_YCrCb = cv2.convertScaleAbs(img_YCrCb)

        return img_YCrCb

    # convert rgb sang hsv
    def convert_hsv(self):
        img_bgr = cv2.imread(self.img_path)
        img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

        return img_hsv


    def gen_mask(self):
        img_rgb = cv2.imread(self.img_path)
        img_argb = self.convert_ARGB()
        img_YCrCb = self.convert_YCrCb()
        img_hsv = self.convert_hsv()

        img_mask = np.zeros((img_rgb.shape[0], img_rgb.shape[1], 1), dtype=bool)
        if self.mode == 1:
            for i in range(img_rgb.shape[0]):
                for j in range(img_rgb.shape[1]):
                    a = int(img_argb[i, j, 3])
                    r = int(img_argb[i, j, 0])
                    g = int(img_argb[i, j, 1])
                    b = int(img_argb[i, j, 2])
                    h = int(img_hsv[i, j, 0])
                    s = float(img_hsv[i, j, 1])
                    v = float(img_hsv[i, j, 2])

                    # print(a, r, g, b, h, s, v)

                    condition_argb = (r > 95) and (g > 40) and (b > 20) and (r > g) and (r > b) and (math.fabs(r - g) > 15) and (a > 15)
                    condition_hsv = (h >= 0) and (h <= 100) and (s >= 0.23*255) and (s <= 0.68*255)

                    # print(condition_argb, condition_hsv)

                    if condition_argb and condition_hsv:
                        img_mask[i, j] = True

        elif self.mode == 2:
            for i in range(img_rgb.shape[0]):
                for j in range(img_rgb.shape[1]):
                    a = int(img_argb[i, j, 3])
                    r = int(img_argb[i, j, 0])
                    g = int(img_argb[i, j, 1])
                    b = int(img_argb[i, j, 2])
                    y = int(img_YCrCb[i, j, 0])
                    Cr = int(img_YCrCb[i, j, 1])
                    Cb = int(img_YCrCb[i, j, 2])

                    condition_argb = (r > 95) and (g > 40) and (b > 20) and (r > g) and (r > b) and (math.fabs(r - g) > 15) and (a > 15)
                    condition_ycrcb = (Cr > 135) and (Cb > 85) and (y > 80) and (Cr <= (1.5862*Cb) + 20) and (Cr >= (0.3448*Cb) + 76.2069) \
                    and (Cr >= (-4.5652*Cb) + 234.5652) and (Cr <= (-1.15*Cb) + 301.75) and (Cr <= (-2.2857*Cb) + 432.85)

                    if condition_argb and condition_ycrcb:
                        img_mask[i, j] = True

        return img_mask

### Predict chế độ 1 rgb.hsv

In [ ]:
import os 

face_photo_path = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/Pratheepan_Dataset/FacePhoto"
folder_face_photo_path_result = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/predict_rgb_hsv/FacePhoto"

family_photo_path = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/Pratheepan_Dataset/FamilyPhoto"
folder_family_photo_path_result = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/predict_rgb_hsv/FamilyPhoto"

for img_path in os.listdir(face_photo_path):
    predict_mode1 = skin_detection(os.path.join(face_photo_path, img_path), 1)
    result = predict_mode1.gen_mask()

    indices = result.astype(int) 
    indices*=255                   
    cv2.imwrite(os.path.join(folder_face_photo_path_result, img_path.split("/")[-1]), indices)
    
for img_path in os.listdir(family_photo_path):
    predict_mode1 = skin_detection(os.path.join(family_photo_path, img_path), 1)
    result = predict_mode1.gen_mask()

    indices = result.astype(int)  
    indices*=255                  
    cv2.imwrite(os.path.join(folder_family_photo_path_result, img_path.split("/")[-1]), indices)

### Predict chế độ 2 - rgb.ycrcb

In [ ]:
import os 

face_photo_path = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/Pratheepan_Dataset/FacePhoto"
folder_face_photo_path_result = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/predict_rgb_ycrcb/FacePhoto"

family_photo_path = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/Pratheepan_Dataset/FamilyPhoto"
folder_family_photo_path_result = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/predict_rgb_ycrcb/FamilyPhoto"

for img_path in os.listdir(face_photo_path):
    predict_mode1 = skin_detection(os.path.join(face_photo_path, img_path), 1)
    result = predict_mode1.gen_mask()

    indices = result.astype(int)  
    indices*=255                  
    cv2.imwrite(os.path.join(folder_face_photo_path_result, img_path.split("/")[-1]), indices)
    
for img_path in os.listdir(family_photo_path):
    predict_mode1 = skin_detection(os.path.join(family_photo_path, img_path), 1)
    result = predict_mode1.gen_mask()

    indices = result.astype(int)  
    indices*=255                  
    cv2.imwrite(os.path.join(folder_family_photo_path_result, img_path.split("/")[-1]), indices)

### Evalution

In [ ]:
class evalution: 
    def __init__(self, img_ground_path, img_predict_path):
        self.img_ground_path = img_ground_path 
        self.img_predict_path = img_predict_path
        self.img_ground = cv2.imread(self.img_ground_path, cv2.IMREAD_GRAYSCALE)
        self.img_predict = cv2.imread(self.img_predict_path, cv2.IMREAD_GRAYSCALE)

    # tính tổng pixel 
    def count_total_pixel(self):
        total_pixel = int(self.img_predict.shape[0]) * int(self.img_predict.shape[1])

        return total_pixel

    # tính pixel da trong predict
    def count_skin_detect(self):
        skin_pixel_count = cv2.countNonZero(self.img_predict)

        return int(skin_pixel_count)

    # tính pixel không phải da trong predict
    def count_not_skin_detect(self):
        total_pixel = self.count_total_pixel()
        skin_pixel_count = self.count_skin_detect()

        ground_pixel_count = total_pixel - skin_pixel_count
        
        return ground_pixel_count

    # tính pixel da trong ground truth 
    def count_skin_gt(self):
        skin_pixel_count = cv2.countNonZero(self.img_ground)

        return int(skin_pixel_count)

    # tính pixel không phải da trong ground truth 
    def count_not_skin_gt(self):
        total_pixel = self.count_total_pixel()
        not_skin_pixel_count = self.count_skin_gt()

        predict_pixel_count = total_pixel - not_skin_pixel_count

        return predict_pixel_count

    # caculate true positive, false positive, true negative, false negative  
    def caculate_predict(self):
        img_gt = self.img_ground > 128
        img_predict = self.img_predict > 128 

        true_positive = np.sum(img_gt & img_predict)
        false_positive = np.sum(~img_gt & img_predict)
        true_negative = np.sum(~img_gt & ~img_predict)
        false_negative = np.sum(img_gt & ~img_predict)

        return true_positive, false_positive, true_negative, false_negative

    # tính giá trị precision
    def caculate_metric(self):
        tp, fp, tn, fn = self.caculate_predict()

        # tính precision 
        precision = (tp/(tp + fp)) * 100
        # tính accuracy 
        accuracy = ((tp + tn)/(tp + fp + tn + fn)) * 100

        return round(precision, 1), round(accuracy, 1)

### Vẽ bảng kết quả

In [ ]:
from prettytable import PrettyTable
import os

# Tạo bảng mới với các tên cột
table = PrettyTable()
table.field_names = ["Sr. No.", "Total of pixel", "Skin pixels detected", "Skin pixels in GT", "NonSkin pixels detected", "NonSkin pixels in GT", \
                    "TP", "FP", "TN", "FN", "Precision", "Accuracy"]

folder_gt_path = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/Ground_Truth/GroundT_FacePhoto"
folder_pre_path = "/content/drive/MyDrive/BareFoot_Thực_tập/human_skin_detection_using_rgb,hvs_and_ycbcr_color_models/dataset/predict_rgb_hsv/FacePhoto"

for i, img_path in enumerate(os.listdir(folder_gt_path)):
    img_name = img_path.split(".")[0]

    if i < 8:
        results = evalution(os.path.join(folder_gt_path, img_name + ".png"), os.path.join(folder_pre_path, img_name + ".jpg"))
        total_pixel = results.count_total_pixel()
        skin_predict = results.count_skin_detect()
        not_skin_predict = results.count_not_skin_detect()
        skin_gt = results.count_skin_gt()
        not_skin_gt = results.count_not_skin_gt()
        tp, fp, tn, fn = results.caculate_predict()
        precision, accuracy = results.caculate_metric()

        table.add_row([i, total_pixel, skin_predict, skin_gt, not_skin_predict, not_skin_gt, tp, fp, tn, fn, precision, accuracy])
    else: 
        break

# In bảng
print(table)